In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import zipfile # ZIPアーカイブを作成するためのクラス

# train.zipの解凍
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/train')

# test.zipの解凍
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/test')


In [ ]:
import numpy as np # みんな大好き numpy（Pythonでの機械学習の計算をより速く、効率的に行えるようにする拡張モジュール）
import pandas as pd
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model


In [ ]:
# trainデータのディレクトリのパスだよ
train_dir = '/kaggle/working/train/train'


In [ ]:
# testデータのディレクトリのパスだよ
test_dir = '/kaggle/working/test/test'


In [ ]:
# 画像のサイズ統一
IMG_SIZE = 64


In [ ]:
# データを読み込む用の関数
def load_data(data_dir,sample_size=1000):
    images = [] # 写真を入れる用のリストちゃん
    labels = [] # その写真のラベル（わんこかにゃんこか）を入れる用のリストちゃん
    files = os.listdir(data_dir)[:sample_size] # listdirはファイルの一覧を得ることができるやつ
#     print(f'（発見） {len(files)} files in {data_dir}')  # デバッグ文（見つけたディレクトリを表示）
    for file in files: # for文でくるくる回してく〜
        img_path = os.path.join(data_dir, file) # ディレクトリのパスとファイル名（写真名）をくっつけて（join）、ファイル（写真）のパスを作る。それをimg_pathに入れる
#         print(f'（ファイル読み込み） {img_path}')  # デバッグ文（読み込んだファイルを表示）
        img = cv2.imread(img_path) # 写真を取得〜
        if img is not None: # もし、写真がNoneじゃなかったら（つまり、写真がきちんとあったら）
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) # その写真のサイズとかを調整〜
            images.append(img) # リストの中に一枚ずつ入れてく〜
            label = 1 if 'dog' in file else 0 # 写真がもし、わんこなら 1, にゃんこなら 0とラベル付して
            labels.append(label) # そのラベルをリストに入れてく〜
        else: #　それ以外（つまり、写真がきちんと読み込めてなかったら）
            print(f'（エラー）ファイル読み込めてねーじゃんか！！ {img_path}')  # デバッグ文（読み込めてないとエラーを吐く）
    return np.array(images) / 255.0, np.array(labels) # かえりち（写真とラベルのリストたち）


In [ ]:
# trainデータとtestデータを読み込んでる
X_train, y_train = load_data(train_dir,sample_size=1000)
X_test, y_test = load_data(test_dir, sample_size=500)


In [ ]:
# print(f'X_train shape: {X_train.shape}')


In [ ]:
# print(f'y_train shape: {y_train.shape}')


In [ ]:
print(f'X_test shape: {X_test.shape}')


In [ ]:
print(f'y_test shape: {y_test.shape}')


In [ ]:
# データ拡張しとこ〜
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
# モデルを作る
def create_model(neuron):
    # ライブラリとか関数とかをインポートする
    Dense = keras.layers.Dense
    Conv2D = keras.layers.Conv2D
    MaxPooling2D = keras.layers.MaxPooling2D
    Flatten = keras.layers.Flatten
    Dropout = keras.layers.Dropout
    
    # モデルを定義する
    model = keras.models.Sequential()
    
    # ここから畳み込みの醍醐味。レイヤー追加してく〜
    
    # まず、32このフィルターと3x3のカーネルサイズを持ってる畳み込みそう。活性化関数はRelu。あと画像の形状の指定。今回は3だからカラー（あか、あお、緑）だよ
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
    # 2x2のプーリング層。空間サイズを半分に
    model.add(MaxPooling2D((2, 2)))
    
    # 次、64このフィルターと3x3のカーネルサイズを持ってる畳み込みそう。活性化関数はRelu。
    model.add(Conv2D(64, (3, 3), activation='relu'))
    # 2x2のプーリング層。空間サイズを半分に
    model.add(MaxPooling2D((2, 2)))
    
    # 次、128このフィルターと3x3のカーネルサイズを持ってる畳み込みそう。活性化関数はRelu。
    model.add(Conv2D(128, (3, 3), activation='relu'))
    # 2x2のプーリング層。空間サイズを半分に
    model.add(MaxPooling2D((2, 2)))
    
    # 全部の結合そうを追加　　　　　　　　
    # Flatten(): 3次元の特徴マップを1次元のベクトルに変換
    model.add(Flatten())
    # neuron数のユニットを持つ全結合層。活性化関数はRelu
    model.add(Dense(neuron, activation='relu'))
    # ドロップアウト正則化。半分のニューロンをランダムに無効。
    model.add(Dropout(0.5))
    # 出力層。シグモイド
    model.add(Dense(1, activation='sigmoid'))

    # モデルをコンパイルする
    # バイナリクロスエントロピー損失関数。Adam。モデルの評価は精度で
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # モデルを返す
    return model


In [ ]:
def save_model(model, filename):
    model.save(filename)
    


In [ ]:
def load_existing_model(filename):
    return load_model(filename)


In [ ]:
# 学習し、テストデータで評価し、スコアを表示する関数
def fit_epoch(neuron, batch, epochs, initial_epoch=0, model_filename='model.h5'):
    if initial_epoch == 0: # もし、モデルを作るのが初めてなら
        # さっき作ったモデル作る関数を実行（ニューロン数送る）
        model = create_model(neuron)
    else: # もうすでに作ったことがあるなら
        model = load_existing_model(model_filename) #モデルを読み込む
        
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # モデルを学習する
    hist = model.fit(datagen.flow(X_train, y_train, batch_size=batch),
                     steps_per_epoch=len(X_train) // batch,
                     validation_data=(X_test, y_test),
                     epochs=epochs,
                     initial_epoch=initial_epoch,
                     verbose=1)
    
    # モデルを評価する。そしてそれをprint
    score = model.evaluate(X_test, y_test, verbose=1)
    print('正解率=', score[1], 'loss=', score[0])
    
    # モデルを保存
    save_model(model, model_filename)
    
    # 学習結果を可視化、グラフ化するよ。ほら、文字全部見るのってだるいじゃん
    
    # trainデータの精度の流れをグラフに
    plt.plot(hist.history['accuracy'])
    # testデータの精度の流れをグラフに
    plt.plot(hist.history['val_accuracy'])
    # グラフのタイトル。あくちゅありー
    plt.title('Accuracy')
    # 凡例
    plt.legend(['train', 'test'], loc='upper left')
    # グラフの表示
    plt.show()

    # trainデータの損失値の流れをグラフに
    plt.plot(hist.history['loss'])
    # testデータの損失値の流れをグラフに
    plt.plot(hist.history['val_loss'])
    # グラフのタイトル。
    plt.title('Loss')
    # 凡例
    plt.legend(['train', 'test'], loc='upper left')
    # グラフの表示
    plt.show()
    


In [ ]:
total_epochs = 30  # 全体のエポック数〜
neuron = 512  # ニューロンの数〜
batch = 8  # バッチサイズ〜
model_filename = 'model.h5'  # モデルのファイル名（モデルの拡張子は.h5だよ）


In [ ]:
# 1回目の学習
fit_epoch(neuron=neuron, batch=batch, epochs=10, initial_epoch=0, model_filename=model_filename)


In [ ]:
# 2回目の学習
fit_epoch(neuron=neuron, batch=batch, epochs=20, initial_epoch=10, model_filename=model_filename)


In [ ]:
# 3回目の学習
fit_epoch(neuron=neuron, batch=batch, epochs=30, initial_epoch=20, model_filename=model_filename)


In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow.keras as keras
from tensorflow.keras.models import load_model

# 定数の設定
IMG_SIZE = 64  # 画像のサイズ
test_dir = '/kaggle/working/test/test'  # テストデータのディレクトリ
model_filename = 'model.h5'  # 学習したモデルのファイル名
output_csv = '/kaggle/working/submission.csv'  # 提出するCSVファイルのパス

# 画像データの読み込み関数
def load_test_data(data_dir):
    images = []
    filenames = os.listdir(data_dir)
    for file in filenames:
        img_path = os.path.join(data_dir, file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
        else:
            print(f'Error reading {img_path}')
    return np.array(images) / 255.0, filenames

# テストデータの読み込み
X_test, test_filenames = load_test_data(test_dir)

# モデルの読み込み
model = load_model(model_filename)

# 予測の実行
predictions = model.predict(X_test)

# 予測結果を0または1に変換
predictions = (predictions > 0.5).astype(int).flatten()

# CSVファイルの作成
output_df = pd.DataFrame({
    'id': [os.path.splitext(f)[0] for f in test_filenames],  # ファイル名から拡張子を除去してIDにする
    'label': predictions
})
output_df.to_csv(output_csv, index=False)

print(f'CSVファイル {output_csv} を作成しました')
